# 本筆記將帶大家熟悉MXNet Gluon
# 此範例使用Fasion MNIST資料集

---

# 索引

1. [選擇Device為GPU0](#1.-選擇Device為GPU0)
2. [小練習：向量相加](#2.-小練習：向量相加)
3. [準備資料: Fashion MNIST dataset](#3.-準備資料:-Fashion-MNIST-dataset)
4. [資料視覺化](#4.-資料視覺化)
5. [建立 mini-batch data generator](#5.-建立-mini-batch-data-generator)
6. [定義模型](#6.-定義模型)
7. [初始化模型權重參數](#7.-初始化模型權重參數)
8. [定義Loss](#8.-定義Loss)
9. [定義Optimizer](#9.-定義Optimizer)
10. [定義Trainer](#10.-定義Trainer)
10. [訓練一個imperative模型](#11.-訓練一個imperative模型)
10. [複合模型與混精度加速](#12.-複合模型與混精度加速)
10. [建立&訓練一個沒有Dense層的CNN網路](#13.-建立&訓練一個沒有Dense層的CNN網路)
---

---

In [ ]:
import mxnet as mx
from mxnet import gluon,autograd
import numpy as np

import gzip
from os.path import join

import matplotlib.pyplot as plt
%matplotlib inline

---

## 1. 選擇Device為GPU0

In [ ]:
ctx = mx.gpu(0) # we'll "flow our tensors" with GPU0

[[返回索引]](#索引)

## 2. 小練習：向量相加

於imperative模式, 我們可使用GPU做向量相加，非常容易。

In [ ]:
from mxnet import nd

In [ ]:
arr=nd.array([1,2,3])

In [ ]:
arr

In [ ]:
arr2=nd.array([1,2,3],ctx=ctx)

In [ ]:
arr2

In [ ]:
arr3=arr.as_in_context(arr2)

In [ ]:
arr3

In [ ]:
arr2+arr3

[[返回索引]](#索引)

---

## 3. 準備資料: Fashion-MNIST dataset

In [ ]:
def trans(data,label):
    return (nd.transpose(data,axes=(2,0,1) ).astype(np.float32) / 255.),label

# 載入資料，並將圖的值除以255，使其範圍落於[0,1]的區間內。
data_train=[d for d in gluon.data.vision.FashionMNIST(root="../datasets/fashion_mnist/", train=True,transform=trans)]
data_test=[d for d in gluon.data.vision.FashionMNIST(root="../datasets/fashion_mnist/", train=False,transform=trans)]

[[返回索引]](#索引)

## 4. 資料視覺化

In [ ]:
fig,axes=plt.subplots(2,5,figsize=(7,3))

for row_axes in axes:
    for col_axis in row_axes:
        # randomly choose a figure to plot
        rand_num=np.random.choice(len(data_train))
        idx=rand_num
        image=data_train[idx][0][0]
        label_image=data_train[idx][1]
        # plot the chosen figure
        col_axis.imshow((image.asnumpy()*255.).astype(np.float32),cmap="gray")
        col_axis.axis("off")
        col_axis.set_title(label_image)

In [ ]:
print("number of train data=\t",len(data_train))
print("number of test data=\t",len(data_test))

[[返回索引]](#索引)

## 5. 建立 mini-batch data generator

In [ ]:
train_gen=gluon.data.DataLoader(data_train,batch_size=128)
test_gen=gluon.data.DataLoader(data_test,batch_size=128)

In [ ]:
# ======= test block =======
# let's double check if the mini-batch generator works as expected
for batch_images,batch_labels in test_gen:
    print(batch_images.shape,batch_labels.shape)
    break
# ======= test block ends =======

[[返回索引]](#索引)

## 6. 定義模型

就如同Keras一樣，於模型建立階段，MXNet Gluon亦可以使用 *Sequential* 的方式逐一疊加網路層。

In [ ]:
from mxnet.gluon import nn

In [ ]:
# ====================================
# exercise: let's build a MLP
# ====================================
# model=nn.Sequential()
# with model.name_scope():
#     model.add( nn.Flatten())
#     model.add( ...
#              )
#     model.add( ...
#              )

In [ ]:
#the following is a simple conv neural network
model=nn.Sequential()

with model.name_scope():
    model.add(nn.Conv2D(64, kernel_size=3, strides=1,padding=1, activation="relu"))
    model.add(nn.Conv2D(64, kernel_size=2, strides=1, activation="relu") )
    model.add(nn.MaxPool2D())
    model.add(nn.Conv2D(128, kernel_size=2, strides=1, activation="relu") )
    model.add(nn.Conv2D(128, kernel_size=2, strides=1, activation="relu") )
    model.add(nn.MaxPool2D())
    model.add(nn.Flatten())
    model.add(nn.Dense(128, activation="relu"))
    model.add(nn.Dense(10))

[[返回索引]](#索引)

## 7. 初始化模型權重參數

In [ ]:
model

In [ ]:
model.collect_params().initialize(mx.init.Xavier(),ctx=ctx) # 權重參數初始化

In [ ]:
# ===================== TEST BLOCK BEGINS ======================================
# Let's check the forward pass, by simply feeding a batch of random images into 
# the model we have just built.
images=np.random.normal(0,1,(32,1,28,28))
images=nd.array(images,ctx=ctx,dtype=np.float32)
print(model(images).shape)
# ===================== TEST BLOCK ENDS ========================================

In [ ]:
# ========================= TEST BLOCK BEGINS =================================================
nd.softmax(model(images)).sum(axis=1) # let's check if the probabilities are conserved if 
                                      # we feed the output to a softmax layer
# ========================= TEST BLOCK ENDS ===================================================

[[返回索引]](#索引)

## 8. 定義Loss

In [ ]:
softmaxCrossEntropyLoss=gluon.loss.SoftmaxCrossEntropyLoss()

[[返回索引]](#索引)

## 9. 定義Optimizer

In [ ]:
opt=mx.optimizer.SGD(momentum=0.9,
                     wd=0.001,
                     learning_rate=0.01)

[[返回索引]](#索引)

## 10. 定義Trainer

A trainer helps us to fetch all the parameters & gradients and update them with the help of the optimizer.

In [ ]:
trainer=gluon.Trainer(params=model.collect_params(),optimizer=opt)

[[返回索引]](#索引)

## 11. 訓練一個imperative模型

In [ ]:
from muon import CreateModel # 給原model添加fit等功能。

In [ ]:
%%time

# 我們利用CreateModel來賦予model新的功能，也就是：fit。
model=CreateModel(model,ctx=ctx,precision=np.float32)

# 初始化模型權重參數
model.collect_params().initialize( mx.init.Xavier(),
                                   ctx=model.ctx,
                                   force_reinit=True)

# 使用 model.fit 訓練模型
model.fit(train_gen,
          test_gen,
          epochs=3,
          print_every=200,
          loss_with_softmax=softmaxCrossEntropyLoss,
          optimizer=opt)

[[返回索引]](#索引)

---

## 12. 複合模型與混精度加速

#### 複合模型與混精度加速
* 若模型是複合模型，利用```model.hybridize()```可將模型由原本的imperative模式切換至symbolic模式。若資料量足夠，一般來說，此切換可將模型訓練速度提升至約2x。
* 當資料與模型權重參數皆使用半精度做儲存時，我們可設定Optimizer是```multi_precision=True```。如此一來，我們即可利用*單半混合精度* (mixed precision)的方式來訓練模型。

#### 補充
1. NVIDIA新的GPU架構(Volta)支持單半混精加速。
2. 單 / 半精度亦即數值使用FP32(4個bytes) / FP16(2個bytes)來儲存。

#### 參考資料
* mixed precision training
    * [NVIDIA's SDK](http://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html#training_mxnet)
    * [Info from NVIDIA's blog](https://devblogs.nvidia.com/mixed-precision-training-deep-neural-networks/)
    * [Paper](https://arxiv.org/pdf/1710.03740.pdf)
* hybridizing (MXNet Gluon)
    * [命令式和符號式混合编程](https://zh.gluon.ai/chapter_computational-performance/hybridize.html#命令式和符号式混合编程)

In [ ]:
from muon import CreateHybridModel # 給複合模型添加fit等功能。

In [ ]:
%%time

# 建立複合(hybrid)模型
layer=nn.HybridSequential()
with layer.name_scope():
    layer.add(nn.Conv2D(64, kernel_size=3, strides=1,padding=1, activation="relu"))
    layer.add(nn.Conv2D(64, kernel_size=2, strides=1, activation="relu") )
    layer.add(nn.MaxPool2D())
    layer.add(nn.Conv2D(128, kernel_size=2, strides=1, activation="relu") )
    layer.add(nn.Conv2D(128, kernel_size=2, strides=1, activation="relu") )
    layer.add(nn.MaxPool2D())
    layer.add(nn.Flatten())
    layer.add(nn.Dense(128, activation="relu"))
    layer.add(nn.Dense(10))

model=CreateHybridModel(layer,ctx=ctx,precision=np.float16)

# 初始化模型權重參數
model.collect_params().initialize(mx.init.Xavier(),ctx=model.ctx)

# 將模型切換至Symbolic模式
model.hybridize()

# 定義Loss和Optimizer
softmaxCrossEntropyLoss=gluon.loss.SoftmaxCrossEntropyLoss()
opt=mx.optimizer.Adam(multi_precision=True)

# 訓練模型
model.fit(train_gen,
          test_gen,
          epochs=3,
          print_every=200,
          loss_with_softmax=softmaxCrossEntropyLoss,
          optimizer=opt)

[[返回索引]](#索引)

---

## 13. 建立&訓練一個沒有Dense層的CNN網路

制定模組```conv_pooling_block```，這樣以後建立網路時，可以重複利用該模組。

除此之外，Conv層之間添加BatchNorm，可使得訓練比較穩定。

In [ ]:
def conv_pooling_block(num_filters, kernel_size=3, padding=1):
    '''A basic building block.'''

    layer=nn.HybridSequential()
    if num_filters!=None:
        with layer.name_scope():        
            layer.add(nn.Conv2D(num_filters, kernel_size=kernel_size, padding=padding))
            layer.add(nn.Activation("relu"))
            layer.add(nn.BatchNorm())
    else:
        with layer.name_scope():        
            layer.add(nn.MaxPool2D())
    return layer

In [ ]:
%%time

# 模型設定
num_filters=[64, 64, None, 128, 128, None] 
config_clf_layers=[(512,5,2),(512,7,3)]

# 根據模型設定，建立複合模型
layer=nn.HybridSequential()
with layer.name_scope():
    layer.add( *[conv_pooling_block(num_filter) for num_filter in num_filters] )    # feature extraction part
    layer.add( *[conv_pooling_block(*config) for config in config_clf_layers]  )    # classification part
    layer.add( nn.Conv2D(10,kernel_size=1) )
    layer.add( nn.Flatten() )

model=CreateHybridModel(layer,ctx=ctx,precision=np.float16)

# 初始化模型權重參數
model.collect_params().initialize(mx.init.Xavier(),ctx=model.ctx)
model.hybridize()

# 定義Loss和Optimizer
softmaxCrossEntropyLoss=gluon.loss.SoftmaxCrossEntropyLoss()
opt=mx.optimizer.Adam(multi_precision=True)

# 訓練模型
model.fit(train_gen,
          test_gen,
          epochs=3,
          print_every=200,
          loss_with_softmax=softmaxCrossEntropyLoss,
          optimizer=opt)

[[返回索引]](#索引)